In [10]:
import math
from collections import Counter



In [11]:
# Step 1: Dataset
data = [
    {'loan': 'Yes', 'tax': 'Yes', 'class': 'Positive'},
    {'loan': 'No',  'tax': 'Yes', 'class': 'Negative'},
    {'loan': 'Yes', 'tax': 'No',  'class': 'Positive'},
    {'loan': 'Yes', 'tax': 'Yes', 'class': 'Positive'},
    {'loan': 'No',  'tax': 'No',  'class': 'Negative'},
    {'loan': 'No',  'tax': 'Yes', 'class': 'Negative'},
    {'loan': 'Yes', 'tax': 'No',  'class': 'Positive'},
    {'loan': 'No',  'tax': 'No',  'class': 'Negative'},
    {'loan': 'Yes', 'tax': 'Yes', 'class': 'Positive'},
]




In [12]:
# Step 2: Entropy
def entropy(subset):
    total = len(subset)
    class_counts = Counter(row['class'] for row in subset)
    ent = 0
    for count in class_counts.values():
        p = count / total
        ent -= p * math.log2(p)
    return ent



In [13]:
# Step 3: Information Gain
def information_gain(data, attribute):
    total_entropy = entropy(data)
    values = set(row[attribute] for row in data)
    weighted_entropy = 0
    for value in values:
        subset = [row for row in data if row[attribute] == value]
        weighted_entropy += len(subset) / len(data) * entropy(subset)
    return total_entropy - weighted_entropy



In [18]:
# Step 4: ID3 Algorithm with clear root node selection
def id3(data, features, depth=0):
    classes = [row['class'] for row in data]

    if classes.count(classes[0]) == len(classes):
        return classes[0]

    if not features:
        return Counter(classes).most_common(1)[0][0]

    # Choose the attribute with the highest info gain
    gains = {attr: information_gain(data, attr) for attr in features}
    best_attr = max(gains, key=gains.get)

    if depth == 0:
        print(f" Root Node Selected: {best_attr} (Info Gain = {gains[best_attr]:.4f})")

    tree = {best_attr: {}}
    for value in set(row[best_attr] for row in data):
        subset = [row for row in data if row[best_attr] == value]
        if not subset:
            tree[best_attr][value] = Counter(classes).most_common(1)[0][0]
        else:
            remaining_features = [f for f in features if f != best_attr]
            tree[best_attr][value] = id3(subset, remaining_features, depth+1)
    return tree



In [19]:
# Step 5: Prediction
def predict(tree, sample):
    if isinstance(tree, str):
        return tree
    attr = next(iter(tree))
    val = sample.get(attr)
    subtree = tree[attr].get(val)
    if not subtree:
        return 'Unknown'
    return predict(subtree, sample)


In [20]:
# Step 6: Evaluation
def evaluate(tree, test_data):
    TP = TN = FP = FN = 0
    for row in test_data:
        pred = predict(tree, row)
        actual = row['class']
        if pred == 'Positive' and actual == 'Positive': TP += 1
        elif pred == 'Negative' and actual == 'Negative': TN += 1
        elif pred == 'Positive' and actual == 'Negative': FP += 1
        elif pred == 'Negative' and actual == 'Positive': FN += 1
    accuracy = (TP + TN) / len(test_data)
    precision = TP / (TP + FP) if (TP + FP) else 0
    recall = TP / (TP + FN) if (TP + FN) else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0
    return accuracy, precision, recall, f1

# Run the full process
features = ['loan', 'tax']
tree = id3(data, features)
acc, prec, rec, f1 = evaluate(tree, data)

# Print tree and metrics
import pprint
print("\n Final Decision Tree:")
pprint.pprint(tree)
print(f"\n Performance:\nAccuracy: {acc:.2f}\nPrecision: {prec:.2f}\nRecall: {rec:.2f}\nF1-Score: {f1:.2f}")


 Root Node Selected: loan (Info Gain = 0.9911)

 Final Decision Tree:
{'loan': {'No': 'Negative', 'Yes': 'Positive'}}

 Performance:
Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1-Score: 1.00
